Before we let these guys, let's test a couple epochs just to get a baseline ...

these are on 1 CPU machines ...

In [1]:
from importlib import reload
import utils
import models_tf_only as models
import numpy as np
from time import time
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping, CSVLogger # maybe more
import json

In [25]:
def run_CNN_model():

    BATCH_SIZE = 512
    EPOCHS = 25

    # choose hyperparameters
    d1 = np.random.randint(0,3)
    d2 = np.random.randint(0,3)
    filters = np.random.randint(50,1001)
    kernel_size = np.random.randint(1,11)
    hidden_units = np.random.randint(25,251)
    dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    trainable_embed = True
    embed_vec = np.random.choice(['glove200', 'glove300', 'numberbatch'])
 


    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    cnn = models.CNN(corpus = corpus, d1 = d1, d2 = d2,
                       batch_size = BATCH_SIZE,
                       filters = filters,
                       kernel_size = kernel_size,
                       hidden_units = hidden_units,
                       dropout_rate = dropout_rate,
                       trainable_embed = trainable_embed)

    cnn.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE, sequence_length = (d1 + d2 + 1), conv = True)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE, sequence_length = (d1 + d2 + 1), conv = True)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/cnn_history_' + right_now + ".csv") # log epochs in case I want to look back later

    
    history = cnn.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                                          validation_data=ug_val, #validation_freq=1,
                                          use_multiprocessing=False, shuffle=True)


    results = {"d1":d1, "d2":d2, "filters":filters, "kernel_size":kernel_size, "hidden_units":hidden_units, "dropout_rate":dropout_rate,
               "embed_vec":embed_vec, "acc":history.history['acc'][-5:], "val_acc":history.history['acc'][-5:],
               "loss":history.history['loss'][-5:], "val_loss":history.history['val_loss'][-5:], "time":(time() - start), 
               "trainable_embed":trainable_embed}

    results = {str(k):str(v) for k,v in results.items()}
    with open("logs/cnn_params_" + right_now + ".json", "w") as f:
        json.dump(results, f)
    
    print("-----------------------------------------------")
    
    
    return results

In [26]:
from importlib import reload
reload(utils)
reload(models)
results = run_CNN_model()

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from glove.6B.300d.txt
Epoch 1/25
156/156 [==============================] - 5s 34ms/step - loss: 1.6805 - acc: 0.5323 - val_loss: 1.2022 - val_acc: 0.6027
Epoch 2/25
156/156 [==============================] - 4s 24ms/step - loss: 1.1601 - acc: 0.6097 - val_loss: 1.1427 - val_acc: 0.6202
Epoch 3/25
156/156 [==============================] - 4s 24ms/step - loss: 1.0945 - acc: 0.6287 - val_loss: 1.1105 - val_acc: 0.6280
Epoch 4/25
156/156 [==============================] - 4s 24ms/step - loss: 1.0532 - acc: 0.6401 - val_loss: 1.0928 - val_acc: 0.6343
Epoch 5/25
156/156 [==============================] - 4s 24ms/step - loss: 1.0226 - acc: 0.6510 - val_loss: 1.0888 - val_acc: 0.6317
Epoch 6/25
156/156 [====

In [24]:
results

{'d1': '2',
 'd2': '2',
 'filters': '910',
 'kernel_size': '8',
 'hidden_units': '186',
 'dropout_rate': '0.552',
 'embed_vec': 'glove200',
 'acc': '[0.27962798, 0.27962798, 0.27962798, 0.27962798, 0.27962798]',
 'val_acc': '[0.27962798, 0.27962798, 0.27962798, 0.27962798, 0.27962798]',
 'loss': '[11.611024901525068, 11.611024901525068, 11.611024901525068, 11.611024901525068, 11.611024901525068]',
 'val_loss': '[11.429241767296425, 11.429241767296425, 11.429241767296425, 11.429241767296425, 11.429241767296425]',
 'time': '78.0381224155426'}

In [30]:
results = {str(k):str(v) for k,v in resutls.items()}

with open("logs/TEST_gpu_cnn_params_.json", "w") as f:
    json.dump(results, f)

on one CPU, with a batch size of 25, we were looking at around 37 (!) minutes for one f'cking epoch (regardless, it seems to be running, so I'm killing it. We'll run them for real with a bigger machine.)

### let's test the LSTM

In [40]:
def run_lstm_model():

    BATCH_SIZE = 100
    EPOCHS = 10

    # choose hyperparameters
    num_layers = np.random.randint(2,16) # lstm layers
    hidden_state_size = np.random.randint(100,400)
    dropout_rate = round(np.random.uniform(0,1,size=None), 3)
    trainable_embed = True
    embed_vec = np.random.choice(['glove200', 'glove300', 'numberbatch'])
    stateful = False
    bidirectional = True


    ## start the timer
    start = time()

    # load the corpus
    corpus = utils.AMI_Corpus(seed = 75, embed_vec = embed_vec)

    lstm = models.LSTMSoftmax(corpus,
                              batch_size = BATCH_SIZE,
                              num_layers = num_layers,
                              dropout_rate = dropout_rate,
                              hidden_state_size = hidden_state_size,
                              stateful = stateful, bidirectional = bidirectional, trainable_embed = trainable_embed)

    lstm.model.compile(optimizer = 'adagrad', metrics = ['acc'], loss = 'categorical_crossentropy')

    # create our generators
    ug_train = utils.UtteranceGenerator(corpus, "train", batch_size = BATCH_SIZE)
    ug_val = utils.UtteranceGenerator(corpus, "val", batch_size = BATCH_SIZE)

    # create keras callbacks
    es = EarlyStopping(monitor='val_loss', patience=5, verbose=0)

    right_now = datetime.now().isoformat() # timestamp
    csv_logger = CSVLogger('logs/lstm_history_' + right_now) # log epochs in case I want to look back later

    # note to self, maybe change validation_steps and validation_freq
    history = lstm.model.fit_generator(ug_train, epochs=EPOCHS, verbose=1, callbacks=[es, csv_logger],
                      validation_data=ug_val, #validation_steps=100, #validation_freq=1,
                      use_multiprocessing=False, shuffle=True)

    results = {"num_layers":num_layers, "hidden_state_size":hidden_state_size,"dropout_rate":dropout_rate,
               "embed_vec":embed_vec, "acc":history.history['acc'][-5:], "val_acc":history.history['acc'][-5:],
               "loss":history.history['loss'][-5:], "val_loss":history.history['val_loss'][-5:], "time":time() - start,
                "trainable_embed":trainable_embed, "stateful":stateful, "bidirectional":bidirectional}
    
    results = {str(k):str(v) for k,v in results.items()}

    with open("logs/lstm_params_" + right_now + ".json", "w") as f:
        json.dump(results, f)
        
    print("-------------------")
    
    return results

We're gonna need a bigger boat! This one estimates at roughly an hour per epoch

In [41]:
reload(utils)
reload(models)
reults = run_lstm_model()

Loading corpus from /home/jake_miller/final/nlp-final-project/data
Begin corpus post-processing ...
Splitting corpus into training and test ...
Creating vocabulary from training set ...
Found 8435 unique words.
Building initial embedding matrix ...
(8437, 300)
loading pretrained vectors from glove.6B.300d.txt
Epoch 1/10
798/798 [==============================] - 69s 87ms/step - loss: 2.0055 - acc: 0.3161 - val_loss: 1.5821 - val_acc: 0.4988
Epoch 2/10
798/798 [==============================] - 64s 80ms/step - loss: 1.5874 - acc: 0.4799 - val_loss: 1.5018 - val_acc: 0.5224
Epoch 3/10
798/798 [==============================] - 64s 80ms/step - loss: 1.4877 - acc: 0.5173 - val_loss: 1.4259 - val_acc: 0.5494
Epoch 4/10
798/798 [==============================] - 63s 79ms/step - loss: 1.4188 - acc: 0.5391 - val_loss: 1.3511 - val_acc: 0.5701
Epoch 5/10
798/798 [==============================] - 64s 80ms/step - loss: 1.3612 - acc: 0.5568 - val_loss: 1.3217 - val_acc: 0.5800
Epoch 6/10
798/798 

Note to self:

```
ValueError: If a RNN is stateful, it needs to know its batch size. Specify the batch size of your input tensors: 
- If using a Sequential model, specify the batch size by passing a `batch_input_shape` argument to your first layer.
- If using the functional API, specify the batch size by passing a `batch_shape` argument to your Input layer.
```

In [35]:
a,b,c = np.random.choice([True, False], 3)

In [36]:
a

False

In [42]:
trainable_embed = np.random.choice([True, False])


In [43]:
num_layers = np.random.randint(2,16) # lstm layers
hidden_state_size = np.random.randint(100,400)
dropout_rate = round(np.random.uniform(0,1,size=None), 3)
embed_vec = np.random.choice(['glove200', 'glove300', 'numberbatch'])
bidirectional = np.random.choice([True, False])
stateful = False
trainable_embed = np.random.choice([True, False])

In [44]:
bidirectional

False